# NBV Classification

> Mendoza, M., Vasquez-Gomez, J. I., Taud, H., Sucar, L. E., & Reta, C. (2019). Supervised Learning of the Next-Best-View for 3D Object Reconstruction. arXiv preprint arXiv:1905.05833.

Notebook configuration

In [1]:
display_dataset = True
display_fwd_pretraining = True
load_weights = False
reading_weights_file = 'weights/paper_param.pth'
saving_weights_file = 'log/current_param.pth'
epochs = 40
batch_size = 256
learning_rate = 0.001
dropout_prob = 0.3

In [2]:
import numpy as np
import csv
import cnbv

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
from torch.autograd import Variable

#from tqdm import tqdm
from cnbv import validation

Check for GPU

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(device)

cuda:0


In [4]:
# params = {'epochs': epochs, 'batch_size': batch_size, 'learning_rate': learning_rate, 'dropout_prob': dropout_prob}

# with open("log/parameters.csv", 'w') as csvfile:
#     fieldnames = params.keys()
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     writer.writerow(params)

## Dataset Preprocessing

Read the pose that corresponds to a class

In [5]:
path_to_pose = 'data/points_in_sphere.txt'
nbv_positions = np.genfromtxt(path_to_pose)

This function converts a class to its corresponding pose

In [6]:
# def getPosition(nbv_class, nbv_positions):
#     return nbv_positions[nbv_class]

In [7]:
# n_views = len(nbv_positions)
# print("Number of sensor poses:", n_views)

Split dataset into training and validation subsets

In [8]:
# path_to_dataset = '/Users/dmitriismirnov/Desktop/NBV/nbv-dataset/training'
path_to_dataset = 'dataset'

In [9]:
# dataset = cnbv.Dataset_NBVC_Folder(dataset_dir=path_to_dataset, device=device)

In [10]:
# dataset = cnbv.Dataset_NBVC_modified(dataset_dir = path_to_dataset,
#                                    transform = transforms.Compose([cnbv.To3DGrid(), cnbv.ToTensor()]))

In [11]:
# train_dataset, test_dataset = dataset.generate_train_test()


In [12]:
# path_to_vol_train

In [13]:
import os

# Train
path_to_vol_train = os.path.join(path_to_dataset, 'vol_train.npy')
path_to_lbl_train = os.path.join(path_to_dataset, 'lbl_train.npy')

train_dataset = cnbv.Dataset_NBVC_Full_numpy(vol_file=path_to_vol_train,
                                            lbl_file=path_to_lbl_train,
                                            transform=transforms.Compose([cnbv.To3DGrid(), cnbv.ToTensor()])
                                            )

train_dataloader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size, drop_last=True)

In [14]:
# Test
path_to_vol_test = os.path.join(path_to_dataset, 'vol_test.npy')
path_to_lbl_test = os.path.join(path_to_dataset, 'lbl_test.npy')

test_dataset = cnbv.Dataset_NBVC_Full_numpy(vol_file = path_to_vol_test,
                                     lbl_file = path_to_lbl_test,
                                     transform=transforms.Compose([cnbv.To3DGrid(), cnbv.ToTensor()])
                                     )

#test_dataloader = DataLoader(dataset=test_dataset, shuffle=True, batch_size=batch_size, drop_last=True)

# Training

In [15]:
model = cnbv.net.NBV_Net_3FC(dropout_prob)
model = model.to(device)

In [16]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
cnbv.train(model, 
      optimizer, 
      train_dataloader, 
      criterion=criterion,
      epochs=5)

NameError: name 'nbvs' is not defined

In [ ]:
# #this is hardcoded for 2-file splits
# train_loader = train_dataloader

# running_loss = 0
# save_after = 100

# history_epoch = []
# history_train_loss = []
# history_validation_loss = []
# history_train_accuracy = []
# history_validation_accuracy = []

# import time
# for e in range(epochs):
#     tic = time.time()
# #     model.train()
    
#     for i, sample in enumerate(train_loader):        
#         # get sample data: images and ground truth keypoints
#         X, y = sample['grid'].to(device), sample['nbv_class'].to(device)        
#         optimizer.zero_grad()
#         output = model(X)
#         loss = criterion(output, y)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
#     print(running_loss)

#     toc = time.time()
    
#     print('time per epoch = ', toc - tic)

## Experiments with fully casting the dataset on the device

In [ ]:
megads_train = torch.utils.data.TensorDataset(
    torch.Tensor(train_dataset.grid_data.reshape(-1,1,32,32,32)).to(device),
    torch.tensor(train_dataset.nbv_class_data, dtype=torch.long).to(device))

In [ ]:
train_dataloader_fullcast = DataLoader(dataset=megads_train, shuffle=True, batch_size=batch_size, drop_last=True)

In [ ]:
del train_dataset
#del train_dataloader

In [ ]:
megads_test = torch.utils.data.TensorDataset(
    torch.Tensor(test_dataset.grid_data.reshape(-1,1,32,32,32)).to(device),
    torch.tensor(test_dataset.nbv_class_data, dtype=torch.long).to(device))

In [ ]:
test_dataloader_fullcast = DataLoader(dataset=megads_test, shuffle=True, batch_size=batch_size, drop_last=True)

In [ ]:
del test_dataset
#del test_dataloader

In [ ]:
enumerate(['cxsc','kschk','tag'])

In [ ]:
"""
#this is hardcoded for 2-file splits
running_loss = 0
save_after = 100

history_epoch = []
history_train_loss = []
history_validation_loss = []
history_train_accuracy = []
history_validation_accuracy = []

import time
for e in range(epochs):
    tic = time.time()
    for i, (X, y) in enumerate(train_dataloader2):        
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y.flatten())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    toc = time.time()
    print('time per epoch = ', toc - tic)
"""

In [ ]:
# learning for our glorious tensor dataset

In [ ]:
# cnbv.validation(net, test_dataloader, criterion, 'cpu')

In [ ]:
# RuntimeError: Expected 5-dimensional input for 5-dimensional weight [10, 1, 3, 3, 3], but got 4-dimensional input of size [16, 32, 32, 32] instead

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history_train_loss)

In [ ]:
plt.plot(history_epoch, history_train_loss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train Loss')
# plt.savefig('log/train_loss.png')
plt.show()

plt.plot(history_epoch, history_train_accuracy)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train set accuracy')
# plt.savefig('log/train_accuracy.png')
plt.show()

plt.plot(history_epoch, history_validation_accuracy)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation set accuracy')
# plt.savefig('log/validation_accuracy.png')
plt.show()

# Test network

In [ ]:
# returns: test images, test predicted keypoints, test ground truth keypoints
test_X, test_outputs, gt_y = net_sample_output()
visualize_output(test_X, test_outputs, np.squeeze(gt_y))

In [ ]:
# save parameters
torch.save(net.state_dict(), saving_weights_file)

In [ ]:
# save metrics

np.save('log/train_loss', history_train_loss)
np.save('log/validation_loss', history_validation_loss)
np.save('log/train_accuracy', history_train_accuracy)
np.save('log/validation_accuracy', history_validation_accuracy)